<a href="https://colab.research.google.com/github/Aloxxa/r_public/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%92%D0%A2%D0%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
!gdown --id 1tDMCSOctLT2ECg9mD5OIdg0kZtw4q9FI

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1tDMCSOctLT2ECg9mD5OIdg0kZtw4q9FI
To: /content/database.db
100% 6.06M/6.06M [00:00<00:00, 137MB/s]


In [157]:
!pip install plotly

In [158]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
import plotly.express as px
import plotly.graph_objects as go
import sqlite3
con = sqlite3.connect('db')
def select(sql):
  return pd.read_sql(sql,con)

In [159]:
import plotly.io as pio
pio.templates.default = "plotly_dark"

In [160]:
query = 'SELECT * FROM pl'
con = sqlite3.connect('database.db')
df_pl = pd.read_sql_query(sql = query, con=con)

In [161]:
df_pl.sample(3)

,client_id,segment,period,product,pl
63440,p0Fy4fsmUA,Стандартный,p_2,РКО,21.869198
25983,21KO3Ucdz2,Массовый,p_1,РКО,2.772379
66743,Q9WY3qp4A4,Верхний,p_2,РКО,42.792709


In [162]:
df_pl['pl'] = df_pl['pl'].round(3)

In [163]:
df_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92609 entries, 0 to 92608
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   client_id  92609 non-null  object 
 1   segment    92609 non-null  object 
 2   period     92609 non-null  object 
 3   product    92609 non-null  object 
 4   pl         92609 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.5+ MB


In [164]:
df_pl.isna().sum().sum()

0

In [165]:
df_pl = df_pl.drop_duplicates()

In [166]:
df_pl.to_sql('df_pl', con, if_exists='replace', index=False)

92609

In [167]:
query = 'SELECT * FROM campaign'
con = sqlite3.connect('database.db')
df_c = pd.read_sql_query(sql = query, con=con)

In [168]:
df_c.sample(3)

,client_id,product,response,manager_id
4881,2v2DaDPe7L,Кредит,Отказ,CM_doxY9cvELI
261,QkuoMueWRF,Кредит,Согласие,CM_fQHTUUUkXc
4083,sEftCR4SLI,Кредит,Отказ,CM_c28lGRRaEI


In [169]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   client_id   5000 non-null   object
 1   product     5000 non-null   object
 2   response    5000 non-null   object
 3   manager_id  5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [170]:
df_c.isna().sum().sum()

0

In [171]:
df_c = df_c.drop_duplicates()

In [172]:
df_c.to_sql('df_c', con, if_exists='replace', index=False)

5000

1) Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.

In [173]:
df_pl.sample(3)

,client_id,segment,period,product,pl
37364,6L0CswZX5f,Массовый,p_1,Эквайринг,94.330
86435,YZ1SW0xauT,Стандартный,p_2,Эквайринг,42.228
91283,I50QVRq20E,Массовый,p_2,Банковская карта,6.381


In [174]:
df_c.sample(3)

,client_id,product,response,manager_id
105,LZGiFrFdYk,Кредит,Недозвон,CM_70HjN0EVqf
3757,RWosSvkdXO,Кредит,Недозвон,CM_c28lGRRaEI
1798,N9v9f9iKSc,Кредит,Согласие,CM_RCwv1r0OHO


In [175]:
df_pl.pivot_table(index='period', columns='product', values='pl', aggfunc='mean')

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,18.573438,41.127913,83.007693,8.015622,27.610645
p_2,16.164158,34.708535,92.359958,7.714323,24.324449


In [176]:
t = df_pl.groupby(['period', 'product']).pl.mean().reset_index(name='mean')
px.histogram(t, x='period', y='mean', color='product', barmode='group', text_auto=True)

In [177]:
t = df_pl.pivot_table(index='period', columns='product', values='pl', aggfunc='mean')
t = t.apply(pd.to_numeric, errors='coerce')
t.pct_change().fillna(0) * 100

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,0.000000,0.000000,0.000000,0.000000,0.000000
p_2,-12.971644,-15.608326,11.266744,-3.758895,-11.901918


In [178]:
t.diff().fillna(0)

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,0.00000,0.000000,0.000000,0.000000,0.000000
p_2,-2.40928,-6.419379,9.352265,-0.301299,-3.286196


In [179]:
t = df_pl.pivot_table(index='period', columns='product', values='pl', aggfunc='count')
t

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,1282,5083,7478,23784,3770
p_2,3720,6198,11264,23732,6298


In [180]:
t = df_pl.groupby(['period', 'product']).size().reset_index(name='count')
px.histogram(t, x='period', y='count', color='product', barmode='group', text_auto=True)

In [181]:
t = df_pl.pivot_table(index='period', columns='product', values='pl', aggfunc='count')
t = t.apply(pd.to_numeric, errors='coerce')
t.pct_change().fillna(0) * 100

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,0.000000,0.000000,0.00000,0.000000,0.000000
p_2,190.171607,21.935865,50.62851,-0.218634,67.055703


In [182]:
t.diff().fillna(0)

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,0.0,0.0,0.0,0.0,0.0
p_2,2438.0,1115.0,3786.0,-52.0,2528.0


In [183]:
result = (df_pl.groupby(['period', 'segment', 'product'])
          .agg(avg_pl=('pl', 'mean'), client_count=('client_id', 'nunique'))
          .reset_index())

print(result)

   period      segment           product      avg_pl  client_count
0     p_1      Верхний  Банковская карта   60.571510            49
1     p_1      Верхний           Депозит  112.474543           254
2     p_1      Верхний            Кредит  236.938717           360
3     p_1      Верхний               РКО   22.847641          1149
4     p_1      Верхний         Эквайринг   80.517197           183
5     p_1     Массовый  Банковская карта   13.310219           895
6     p_1     Массовый           Депозит   28.747543          3537
7     p_1     Массовый            Кредит   59.219406          5247
8     p_1     Массовый               РКО    5.795116         16728
9     p_1     Массовый         Эквайринг   19.627603          2625
10    p_1  Стандартный  Банковская карта   26.421592           338
11    p_1  Стандартный           Депозит   60.994265          1292
12    p_1  Стандартный            Кредит  120.101213          1871
13    p_1  Стандартный               РКО   11.418811          

In [184]:
t = df_pl.pivot_table(index=['period', 'segment'], columns='product', values='pl', aggfunc=['count', 'mean'])
t = t.round(3)
t

count                                  \
product            Банковская карта Депозит Кредит    РКО Эквайринг   
period segment                                                        
p_1    Верхний                   49     254    360   1149       183   
       Массовый                 895    3537   5247  16728      2625   
       Стандартный              338    1292   1871   5907       962   
p_2    Верхний                  167     283    548   1162       284   
       Массовый                2636    4345   7845  16700      4424   
       Стандартный              917    1570   2871   5870      1590   

                               mean                                      
product            Банковская карта  Депозит   Кредит     РКО Эквайринг  
period segment                                                           
p_1    Верхний               60.572  112.475  236.939  22.848    80.517  
       Массовый              13.310   28.748   59.219   5.795    19.628  
       Стандартный           26.422   60.994  120.101  11.419    39.330  
p_2    Верхний               51.770   99.261  248.192  22.064    72.969  
       Массовый              11.727   25.220   67.110   5.567    17.348  
       Стандартный           22.436   49.333  131.612  10.984    35.046

2) Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.

In [185]:
t = df_pl.pivot_table(index=['period'], columns='segment', values='pl', aggfunc=['count', 'mean'])
t = t.round(3)
t

count                         mean                     
segment Верхний Массовый Стандартный Верхний Массовый Стандартный
period                                                           
p_1        1995    29032       10370  79.108   19.729      40.283
p_2        2444    35950       12818  89.651   23.273      46.503

3) Оценить отток клиентов с продуктов в периоде p_2.

In [186]:
t = df_pl.pivot_table(index='period', columns='product', values='pl', aggfunc='count')
t = t.apply(pd.to_numeric, errors='coerce')
t.pct_change().fillna(0) * 100

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,0.000000,0.000000,0.00000,0.000000,0.000000
p_2,190.171607,21.935865,50.62851,-0.218634,67.055703


In [187]:
t.diff().fillna(0)

product,Банковская карта,Депозит,Кредит,РКО,Эквайринг
period,,,,,
p_1,0.0,0.0,0.0,0.0,0.0
p_2,2438.0,1115.0,3786.0,-52.0,2528.0


4) Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании

In [188]:
df_pl.sample(3)

,client_id,segment,period,product,pl
80830,6AuTHKeLRO,Верхний,p_2,Депозит,104.842
18130,fX3U0bczbE,Стандартный,p_1,РКО,2.131
72909,MQ2HtKv9nK,Массовый,p_2,РКО,9.086


In [189]:
df_c.sample(3)

,client_id,product,response,manager_id
3192,o53h4j8Hxy,Кредит,Недозвон,CM_6ofody41cY
3965,33bfB9K8N2,Кредит,Согласие,CM_c28lGRRaEI
1312,b4vIcIaS1U,Кредит,Согласие,CM_fQHTUUUkXc


In [190]:
t = df_pl.groupby('client_id').pl.mean().reset_index()
t.sample(2)

,client_id,pl
10645,QSDnhegRxY,23.979
10786,QnPmi6FVnh,48.149


In [191]:
df = pd.merge(df_pl, df_c, how='inner', on='client_id')
df.sample(2)

,client_id,segment,period,product_x,pl,product_y,response,manager_id
10570,yHq6EjSvOU,Массовый,p_2,РКО,0.178,Кредит,Не обработано менеджером,CM_eE0WMK1vVW
14059,PWSEXRJXBd,Массовый,p_2,Эквайринг,0.605,Кредит,Отказ,CM_EiBeeRMhdA


In [192]:
df_c

,client_id,product,response,manager_id
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA
1,HkhuPN1KG7,Кредит,Не обработано менеджером,CM_PzEEav4ic0
2,x1rEdC5fys,Кредит,Согласие,CM_RCwv1r0OHO
3,fx0acVADvj,Кредит,Согласие,CM_nJ7j5pAcMB
4,nDNNydkKQR,Кредит,Согласие,CM_EiBeeRMhdA
...,...,...,...,...
4995,pbL8qc9JQJ,Кредит,Отказ,CM_doxY9cvELI
4996,0u4KYTNs7g,Кредит,Не обработано менеджером,CM_9vMKq9aqw9
4997,forPIbFbFc,Кредит,Отказ,CM_eE0WMK1vVW
4998,TEHS2F9oXq,Кредит,Отказ,CM_nJ7j5pAcMB


In [193]:
df_c['response'].value_counts(normalize=True).reset_index()

,response,proportion
0,Согласие,0.5632
1,Недозвон,0.1498
2,Отказ,0.1434
3,Не обработано менеджером,0.0968
4,Перезвонить,0.0468


In [194]:
t = df[df['response']=='Не обработано менеджером']['manager_id']
t.value_counts().reset_index()

,manager_id,count
0,CM_lKxhD73Z6d,130
1,CM_2755B4KTu4,107
2,CM_pMFgOhtUKD,106
3,CM_R7BdTh0Svt,103
4,CM_6ofody41cY,102
5,CM_c28lGRRaEI,102
6,CM_RCwv1r0OHO,101
7,CM_wjb2ArrpAw,98
8,CM_70HjN0EVqf,95
9,CM_onY6ha8qHv,95


In [195]:
df['gg'] = df['response'].apply(lambda x: 1 if 'Согласие'==x else 0)

In [196]:
df.groupby('gg').pl.mean().reset_index()

,gg,pl
0,0,27.721101
1,1,34.853853


In [197]:
df.groupby('manager_id').agg({'pl': ['sum', 'count'], 'gg': 'mean'}).reset_index().sort_values(by=('pl', 'sum'), ascending=False).rename(columns={'pl': 'total_pl', 'count': 'total_clients', 'gg': 'gg_coef'})

manager_id   total_pl                 gg_coef
                         sum total_clients      mean
0   CM_2755B4KTu4  33358.938           967  0.642192
17  CM_pMFgOhtUKD  32451.323           930  0.579570
8   CM_c28lGRRaEI  32285.017           950  0.594737
5   CM_PzEEav4ic0  31783.189           947  0.582893
4   CM_EiBeeRMhdA  31141.331           992  0.590726
18  CM_wjb2ArrpAw  30827.943           933  0.592712
6   CM_R7BdTh0Svt  30757.158           937  0.609392
1   CM_6ofody41cY  30484.540           966  0.556936
10  CM_doxY9cvELI  30478.236           919  0.586507
14  CM_lKxhD73Z6d  30080.582           969  0.601651
13  CM_gHP4ykbzTj  30030.612           941  0.623804
2   CM_70HjN0EVqf  29887.646           929  0.532831
19  CM_zShYeVNFi0  29852.590           950  0.624211
7   CM_RCwv1r0OHO  29492.810           981  0.607543
11  CM_eE0WMK1vVW  29180.327           958  0.611691
12  CM_fQHTUUUkXc  28067.714           935  0.605348
16  CM_onY6ha8qHv  27988.064           859  0.547148
9   CM_dcTdLjqH2x  26690.978           888  0.615991
3   CM_9vMKq9aqw9  26459.571           879  0.560865
15  CM_nJ7j5pAcMB  25766.098           854  0.600703

5) Оценить эффективность маркетинговой кампании.

In [198]:
periods = df_pl['period'].unique()
period_before_campaign = min(periods)
period_after_campaign = max(periods)

In [199]:
df_before = df_pl[df_pl['period'] == period_before_campaign]
df_after = df_pl[df_pl['period'] == period_after_campaign]

In [200]:
clients_in_campaign = df_c['client_id'].unique()

In [201]:
df_before_campaign = df_before[df_before['client_id'].isin(clients_in_campaign)]
df_after_campaign = df_after[df_after['client_id'].isin(clients_in_campaign)]

In [202]:
sum_before_campaign = df_before_campaign['pl'].sum()
sum_after_campaign = df_after_campaign['pl'].sum()

In [203]:
change = sum_after_campaign - sum_before_campaign
print('эффект', change)

эффект 387064.843
